# The Trouble With Tethers

While orbital rings are cool on their own, they're pretty useless without being able to connect them to the ground; that's where tethers come in. These connect to the ring and run out to ground stations on the sides. Additionally, tethers connect rings where they intersect, but aside from being shorter than the ones going to the ground, there isn't anything particularly interesting about them; we'll disregard them for this chapter.

## Goals

What are we trying to accomplish with tethers? Well, we need them to hang from the ring and then be able to attach them to the ground. They should be able to hold up any cabling that needs to run between the ring and the ground -- electricity and fiber being the notable ones -- as well as providing something for the climbers to hold onto in order to get up and down, and account for the mass of the climber and payload.

We also never want single points of failure, and to build safety factors into everything we do. This means that tethers are never independent units; we group them into bundles to meet our goals. The design of bundles will be discussed later in this chapter.

## Length

Tether length is one of the most important factors to consider, and it can vary from about 175km to 400km. Below this range, it would be running nearly directly down from the ring; above this range, the mass of the tether becomes impractical even for the biggest rings.

While this is quite long, it's relatively easy to make tethers of this length using readily-available materials. This is especially true because of one trick: tethers don't need to be of uniform thickness.

## Tapering

While the tether is hanging from the ring, we actually need to think about this from the ground and work our way up. We imagine a short length -- say, a meter -- of our material of choice. The diameter of that section of tether needs to be enough to hold its own weight and the weight of whatever payload we wish to carry. Then we imagine a meter of tether above that: it needs to hold its own weight, and the weight of everything below it. We do this all the way up to the orbital ring.

What we find is that the bottom needs to only be thick enough to hold the payload, but the top needs to hold the entire weight of the tether plus the payload. This is an exponential taper, which is why length is such a big factor: the longer it is, the bigger the top needs to be.

XXX: Add interactive plot

## Material

The ideal material would have the following properties:

- A high tensile strength relative to its density
- Readily available
- Able to carry power
- Stable through thermal cycling
- Unaffected by UV light

Unfortunately, no such ideal material exists, but we don't need to get *all* of these properties to be successful. Steel, aluminum, and titanium have too low a tensile strength to density ratio to be viable; they would work, but the tethers would simply weigh far too much.

This leaves us with fibers, of which three contenders are worth considering: Kevlar, Zylon, and Spectra.

### Kevlar

Kevlar has a very high tensile strength relative to its density, leading to quite lightweight and thin tethers. Additionally, it's very readily available and plenty thermally stable for our purposes. It can't carry power, which is unfortunate but not a dealbreaker, as we can always attach external wiring to the tethers.

The biggest problem is that Kevlar's tensile strength is degraded heavily and quickly by UV light. This would make it a non-option for our purposes, but there are treatments for the fibers that can substantially reduce this impact.

### Zylon

Zylon is pretty comparable to Kevlar in most regards, with a few notable exceptions:

It has a slightly higher strength-to-density ratio (not enough to make a significant impact, however.) It can also be plated with a conductive material, although it's not known if this can be done in a way that would withstand a crawler riding along it; testing would need to be done for this.

And it's substantially worse at handling UV exposure than Kevlar is, even in its untreated form. It needs to be sheathed in some form, which may or may not be possible for our application. Given that the climber needs to be in mechanical contact with the tether, this is likely to cause problems for our purposes, and likely negates the ability to plate it with a conductive material, if nothing else.

### Spectra

Spectra has a superior strength-to-density ratio to both Kevlar and Zylon, is extremely stable in terms of UV and thermal cycling, and has excellent thermal stability. It can't be made to carry power, but that is a fairly minor tradeoff.

## Bundle Design

Depending on the scale of the ring we're connecting to, we're going to use different designs of bundles. If we have fully-deployed rings where there's major fiber backhaul connecting around the world, we may have one or more tethers whose primary purpose is to just carry those optical fibers. The same is true of rings which provide substantial power production.

However, for simplicity we'll talk about two types of bundles: the simple kind we'd use while bootstrapping our rings, and the complex kind we'd use for advanced rings.

### Simple Bundle

While building our initial ring, the top priority will be keeping mass down, as these tether bundles will need to be launched into space on rockets. As such, a simple bundle will likely consist of:

- Two tethers each capable of carrying a 1000kg payload with a safety factor of 2
- One tether with copper wire carrying high-voltage DC (HVDC) power

HVDC may seem like a strange choice here, but this allows us to minimize the current on the wire, and thus the losses. This means our wire can be much thinner and -- more importantly -- lighter. The wire would be attached to the tether at regular intervals to reduce the stress gravity puts on it.

While we could climb on both bearing tethers with a 2000kg payload, this would mean that if either tether fails, both would fail. Instead, it would be safer to keep it to a 1000kg payload; either tether could hold the whole thing in the event of a failure. (XXX: Calculate and discuss the forces that would be experienced by the remaining tether if one tether fails.)

There is a single point of failure on the tether carrying power. However, this would be under significantly less stress than the other tethers, and would not lead to catastrophic failure in the event that the tether fails. Mechanical brakes can be used to slow the descent of the crawler. (XXX: Show the math on this.)

### Complex Bundle

Once our orbital rings grow, we'll need to be able to carry passengers, large volumes of cargo, substantial power, as well as fiber optics. Due to the addition of passengers, we need to more seriously consider our safety factors, redundancy, and monitoring.

- Five tethers each capable of carrying a 25000kg payload with a safety factor of 3
- Three tethers with copper wire carrying HVDC and sheathed fiber optical cables

In such a configuration, we could carry a 75000kg payload and lose two load-bearing tether in transit without suffering catastrophic loss. The redundant power and fiber tethers are primarily to give additional time to maintenance personnel to remediate the issue(s); with a setup like this, these tethers may be providing a substantial portion of the region's power.

It may also be desirable to use multiple bundles to both ride on top of them rather than crawling below them, and to increase the maximum capacity.